In [47]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path

### Прочитаем все доступные csv с информацией о клиентах банка и сохраним их в датафреймы

In [48]:
PATH = Path("datasets")
EXT = "*.csv"

# dictionary holding all the files DataFrames with the format {"filename": file_DataFrame}
files_dfs = {}

# recursive search for csv files in PATH folder and subfolders 
for csv_file in PATH.rglob(EXT):
    filename = csv_file.name     # get the filename 
    df = pd.read_csv(csv_file)   # read the csv file as a DataFrame
    files_dfs[filename] = df     # add the DataFrame to the dictionary

In [49]:
for dataset in files_dfs:
    display(dataset)

'D_clients.csv'

'D_close_loan.csv'

'D_job.csv'

'D_last_credit.csv'

'D_loan.csv'

'D_pens.csv'

'D_salary.csv'

'D_target.csv'

'D_work.csv'

In [50]:
column_names = {}
res_list = []

for data in files_dfs:
    column_names.update({data:files_dfs[data].columns.tolist()})

for col in column_names.values():
    for col_name in col:
        res_list.append(col_name)


In [51]:
df_clients = pd.read_csv('datasets/D_clients.csv')
df_close_loan = pd.read_csv('datasets/D_close_loan.csv')
df_job = pd.read_csv('datasets/D_job.csv')
df_last_credit = pd.read_csv('datasets/D_last_credit.csv')
df_loan = pd.read_csv('datasets/D_loan.csv')
df_pens = pd.read_csv('datasets/D_pens.csv')
df_salary = pd.read_csv('datasets/D_salary.csv')
df_target = pd.read_csv('datasets/D_target.csv')
df_work = pd.read_csv('datasets/D_work.csv')

### Работаем с информацией о кредитах

Собираем нужную информацию в один датафрейм из двух с помощью группировок 

In [52]:
df_all_loans = df_loan.merge(df_close_loan, 'outer', 'ID_LOAN')
df_all_loans

,ID_LOAN,ID_CLIENT,CLOSED_FL
0,1753790658,106804370,1
1,1753790659,106804371,1
2,1753790660,106804372,1
3,1753790661,106804372,0
4,1753790662,106804373,1
...,...,...,...
21121,1753811779,106819588,1
21122,1753811780,106819589,0
21123,1753811781,106819590,0
21124,1753811782,106819591,0


In [53]:
len(df_all_loans['ID_CLIENT'].unique())

15223

In [54]:
counts = df_all_loans['ID_CLIENT'].value_counts() 
df_all_loans['LOAN_NUM_TOTAL'] = df_all_loans.groupby(['ID_CLIENT'])['ID_LOAN'].transform('count')
df_all_loans['LOAN_NUM_CLOSED'] =  df_all_loans.groupby(df_all_loans['ID_CLIENT'])['CLOSED_FL'].transform('sum')
df_all_loans

,ID_LOAN,ID_CLIENT,CLOSED_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED
0,1753790658,106804370,1,1,1
1,1753790659,106804371,1,1,1
2,1753790660,106804372,1,2,1
3,1753790661,106804372,0,2,1
4,1753790662,106804373,1,1,1
...,...,...,...,...,...
21121,1753811779,106819588,1,3,2
21122,1753811780,106819589,0,1,0
21123,1753811781,106819590,0,1,0
21124,1753811782,106819591,0,1,0


In [55]:
df_all_loans['LOAN_NUM_TOTAL'].unique()

array([ 1,  2,  3,  4,  5,  6, 11,  7,  8], dtype=int64)

In [56]:
df_all_loans['LOAN_NUM_CLOSED'].unique()

array([ 1,  2,  0,  3,  4,  5,  6, 11,  7,  8], dtype=int64)

In [57]:
df_all_loans.groupby(df_all_loans['ID_CLIENT'])['CLOSED_FL'].sum()

ID_CLIENT
106804370    1
106804371    1
106804372    1
106804373    1
106804374    1
            ..
106819588    2
106819589    0
106819590    0
106819591    0
106819592    0
Name: CLOSED_FL, Length: 15223, dtype: int64

### Собираем нужные столбцы в целевой датафрейм

In [58]:
df_clients = df_clients.rename(columns={'ID':'ID_CLIENT'})
df = df_clients.merge(df_target, 'outer', 'ID_CLIENT')
df = df.merge(df_job, 'outer', 'ID_CLIENT')
df = df.merge(df_last_credit, 'outer', 'ID_CLIENT')
df = df.merge(df_all_loans[['LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'ID_CLIENT']], 'outer', 'ID_CLIENT')
df = df.merge(df_salary, 'outer', 'ID_CLIENT')

Попросмотрим на пропуски в данных:

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22312 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                22312 non-null  int64  
 1   AGE                      22312 non-null  int64  
 2   GENDER                   22312 non-null  int64  
 3   EDUCATION                22312 non-null  object 
 4   MARITAL_STATUS           22312 non-null  object 
 5   CHILD_TOTAL              22312 non-null  int64  
 6   DEPENDANTS               22312 non-null  int64  
 7   SOCSTATUS_WORK_FL        22312 non-null  int64  
 8   SOCSTATUS_PENS_FL        22312 non-null  int64  
 9   REG_ADDRESS_PROVINCE     22312 non-null  object 
 10  FACT_ADDRESS_PROVINCE    22312 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  22312 non-null  object 
 12  FL_PRESENCE_FL           22312 non-null  int64  
 13  OWN_AUTO                 22312 non-null  int64  
 14  AGREEMENT_RK          

Пропусков довольно много, информация из разных табличек не очень хорошо стыкуется между собой.

Первым делом избавимся от дубликатов в данных

In [60]:
df = df.drop_duplicates(keep = 'first')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                16000 non-null  int64  
 1   AGE                      16000 non-null  int64  
 2   GENDER                   16000 non-null  int64  
 3   EDUCATION                16000 non-null  object 
 4   MARITAL_STATUS           16000 non-null  object 
 5   CHILD_TOTAL              16000 non-null  int64  
 6   DEPENDANTS               16000 non-null  int64  
 7   SOCSTATUS_WORK_FL        16000 non-null  int64  
 8   SOCSTATUS_PENS_FL        16000 non-null  int64  
 9   REG_ADDRESS_PROVINCE     16000 non-null  object 
 10  FACT_ADDRESS_PROVINCE    16000 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  16000 non-null  object 
 12  FL_PRESENCE_FL           16000 non-null  int64  
 13  OWN_AUTO                 16000 non-null  int64  
 14  AGREEMENT_RK             15

Ручная проверка того, что информация правильно собралась из разных таблиц по одному из клиентов

In [61]:
df.loc[(df['ID_CLIENT'] == 106804372)]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,GEN_TITLE,JOB_DIR,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FAMILY_INCOME,PERSONAL_INCOME
5244,106804372,52,1,Неполное среднее,Состою в браке,4,0,1,0,Иркутская область,...,Специалист,Участие в основ. деятельности,84.0,33126.0,12.0,4000.0,2.0,1.0,от 10000 до 20000 руб.,9000.0


Убираем все данные, в которых есть пропуски в целевой переменной

In [62]:
df = df.loc[df['TARGET'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                15223 non-null  int64  
 1   AGE                      15223 non-null  int64  
 2   GENDER                   15223 non-null  int64  
 3   EDUCATION                15223 non-null  object 
 4   MARITAL_STATUS           15223 non-null  object 
 5   CHILD_TOTAL              15223 non-null  int64  
 6   DEPENDANTS               15223 non-null  int64  
 7   SOCSTATUS_WORK_FL        15223 non-null  int64  
 8   SOCSTATUS_PENS_FL        15223 non-null  int64  
 9   REG_ADDRESS_PROVINCE     15223 non-null  object 
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object 
 12  FL_PRESENCE_FL           15223 non-null  int64  
 13  OWN_AUTO                 15223 non-null  int64  
 14  AGREEMENT_RK             15

In [63]:
df[['GEN_INDUSTRY', 'GEN_TITLE', 'JOB_DIR']]

,GEN_INDUSTRY,GEN_TITLE,JOB_DIR
0,Другие сферы,Работник сферы услуг,Участие в основ. деятельности
1,Торговля,Специалист,Участие в основ. деятельности
2,Другие сферы,Руководитель высшего звена,Участие в основ. деятельности
3,Государственная служба,Специалист,Участие в основ. деятельности
4,Другие сферы,Специалист,Участие в основ. деятельности
...,...,...,...
22306,Сборочные производства,Специалист,Участие в основ. деятельности
22308,Торговля,Специалист,Участие в основ. деятельности
22309,Торговля,Специалист,Участие в основ. деятельности
22310,Транспорт,Специалист,Участие в основ. деятельности


Видим  существенные пропуски в столбцах GEN_INDUSTRY, GEN_TITLE, JOB_DIR. Это категориальные данные, но пропуски будет очень трудно восстановить. 

Как вариант можно было бы заполнить самым часто встречаемым значением по каждому столбцу или обучить модель предсказывать пропуски, но для данного задания просто удалим эти столбцы.

Их нет в списке обязательных для итогового файла, и у нас итак довольно много информации о клиентах банка.

### Необходимые столбцы:

- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
- AGE — возраст клиента;
- SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- CHILD_TOTAL — количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- PERSONAL_INCOME — личный доход клиента (в рублях);
- LOAN_NUM_TOTAL — количество ссуд клиента;
- LOAN_NUM_CLOSED — количество погашенных ссуд клиента.

In [64]:
df = df.drop(['GEN_INDUSTRY', 'GEN_TITLE', 'JOB_DIR'], axis=1)
df['WORK_TIME'].fillna(df['WORK_TIME'].median(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID_CLIENT                15223 non-null  int64  
 1   AGE                      15223 non-null  int64  
 2   GENDER                   15223 non-null  int64  
 3   EDUCATION                15223 non-null  object 
 4   MARITAL_STATUS           15223 non-null  object 
 5   CHILD_TOTAL              15223 non-null  int64  
 6   DEPENDANTS               15223 non-null  int64  
 7   SOCSTATUS_WORK_FL        15223 non-null  int64  
 8   SOCSTATUS_PENS_FL        15223 non-null  int64  
 9   REG_ADDRESS_PROVINCE     15223 non-null  object 
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object 
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object 
 12  FL_PRESENCE_FL           15223 non-null  int64  
 13  OWN_AUTO                 15223 non-null  int64  
 14  AGREEMENT_RK             15

Переводим чилсовые столбцы обратно к формату integer

In [65]:
df[['AGREEMENT_RK', 'TARGET', 'WORK_TIME', 'CREDIT', 
    'TERM', 'FST_PAYMENT', 'LOAN_NUM_TOTAL',
    'LOAN_NUM_CLOSED', 'PERSONAL_INCOME']] = df[['AGREEMENT_RK', 'TARGET', 
                              'WORK_TIME', 'CREDIT', 'TERM', 
                              'FST_PAYMENT', 'LOAN_NUM_TOTAL', 
                              'LOAN_NUM_CLOSED', 'PERSONAL_INCOME']].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15223 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID_CLIENT                15223 non-null  int64 
 1   AGE                      15223 non-null  int64 
 2   GENDER                   15223 non-null  int64 
 3   EDUCATION                15223 non-null  object
 4   MARITAL_STATUS           15223 non-null  object
 5   CHILD_TOTAL              15223 non-null  int64 
 6   DEPENDANTS               15223 non-null  int64 
 7   SOCSTATUS_WORK_FL        15223 non-null  int64 
 8   SOCSTATUS_PENS_FL        15223 non-null  int64 
 9   REG_ADDRESS_PROVINCE     15223 non-null  object
 10  FACT_ADDRESS_PROVINCE    15223 non-null  object
 11  POSTAL_ADDRESS_PROVINCE  15223 non-null  object
 12  FL_PRESENCE_FL           15223 non-null  int64 
 13  OWN_AUTO                 15223 non-null  int64 
 14  AGREEMENT_RK             15223 non-null  in

Попробуем более пристально посмотреть на данные и найти странные неувязки

In [66]:
df.describe()

,ID_CLIENT,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,FL_PRESENCE_FL,OWN_AUTO,AGREEMENT_RK,TARGET,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,PERSONAL_INCOME
count,1.522300e+04,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,1.522300e+04,15223.000000,1.522300e+04,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000
mean,1.068120e+08,40.406096,0.654536,1.099389,0.645208,0.909610,0.134468,0.309794,0.116337,6.540183e+07,0.119030,2.702661e+02,14667.934441,8.101031,3398.548643,1.387769,0.751889,13853.836300
std,4.394646e+03,11.601068,0.475535,0.995411,0.812252,0.286748,0.341165,0.462424,0.320844,4.568181e+06,0.323835,2.324433e+04,12147.874140,4.094090,5158.109298,0.793566,0.988499,9015.467619
min,1.068044e+08,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.991015e+07,0.000000,1.000000e+00,2000.000000,3.000000,0.000000,1.000000,0.000000,24.000000
25%,1.068082e+08,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6.192092e+07,0.000000,2.400000e+01,6500.000000,6.000000,1000.000000,1.000000,0.000000,8000.000000
50%,1.068120e+08,39.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6.437100e+07,0.000000,4.800000e+01,11550.000000,6.000000,2000.000000,1.000000,0.000000,12000.000000
75%,1.068158e+08,50.000000,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,6.708802e+07,0.000000,9.600000e+01,19170.000000,10.000000,4000.000000,2.000000,1.000000,17000.000000
max,1.068196e+08,67.000000,1.000000,10.000000,7.000000,1.000000,1.000000,1.000000,2.000000,7.529224e+07,1.000000,2.867959e+06,119700.000000,36.000000,140000.000000,11.000000,11.000000,250000.000000


In [67]:
df['PERSONAL_INCOME'].unique()

array([ 25000,  10000,  30000,  15000,   7800,  12000,   8000,   7000,
         9000,   4900,  20000,   5500,  55000,   8500,   5000,   6500,
        27000,   6000,  10500,  13000,  11000,  21000,  23000,  22000,
         7500,  12500,  17500,  18000,  40000,  14000,  17000,  16000,
         8700,  11500,  35000,   8600,  45000,   9340,   8200,   6700,
        80000,  12800,  24000,   5200,  12650,   7100,  29000,   9330,
         7850,   4300,  13200,   9500,  34000,   4000,   5582,  19000,
         5400,   5800,   7300,   7600,   9700,  15600,  20500,   3500,
         9800,  36000,  28000,   4800,   4600,  17404,  15500,   3000,
         8100,  26000,   6200, 100000,  24500,  13500,   4500,   5600,
        12900,   5350,   9100,  22955,   7200,  18600,   5460,  18500,
        43000,  10200,   6800,   8350,  11640,   9600, 150000,   6400,
         7900,  17800,   5300,   8400,  10400,   5446,  50000,   7640,
         8900,   4700,  18700,  10300,   5100,   5900,  12600,  19500,
      

Удаляем строчку с зарплатой 24 рубля

In [68]:
df = df.loc[df['PERSONAL_INCOME'] != 24]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15222 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID_CLIENT                15222 non-null  int64 
 1   AGE                      15222 non-null  int64 
 2   GENDER                   15222 non-null  int64 
 3   EDUCATION                15222 non-null  object
 4   MARITAL_STATUS           15222 non-null  object
 5   CHILD_TOTAL              15222 non-null  int64 
 6   DEPENDANTS               15222 non-null  int64 
 7   SOCSTATUS_WORK_FL        15222 non-null  int64 
 8   SOCSTATUS_PENS_FL        15222 non-null  int64 
 9   REG_ADDRESS_PROVINCE     15222 non-null  object
 10  FACT_ADDRESS_PROVINCE    15222 non-null  object
 11  POSTAL_ADDRESS_PROVINCE  15222 non-null  object
 12  FL_PRESENCE_FL           15222 non-null  int64 
 13  OWN_AUTO                 15222 non-null  int64 
 14  AGREEMENT_RK             15222 non-null  in

In [69]:
df['AGE'].unique()

array([42, 28, 64, 54, 26, 63, 32, 52, 47, 23, 31, 25, 37, 46, 62, 50, 30,
       41, 27, 61, 38, 51, 58, 35, 45, 56, 36, 55, 29, 39, 43, 44, 34, 40,
       22, 57, 59, 48, 49, 33, 53, 60, 24, 67, 66, 65, 21], dtype=int64)

In [70]:
df['WORK_TIME'].unique()

array([      3,       5,     360,      12,      48,      96,     217,
            10,      18,     239,       8,      36,      60,     216,
            30,      14,      54,     132,      72,       6,     120,
            84,     504,      90,      29,      24,     108,      40,
           144,      50,     408,      78,     192,     180,     350,
            16,     156,      80,      69,      59,     301,     168,
           228,     197,      20,     276,     300,     384,     288,
            19,     138,       7,     372,     264,      11,     252,
            15,     242,       9,     245,      26,     396,     240,
            42,     204,     150,      13,      39,      49,      64,
           420,     324,      71,      65,      22,     390,      23,
             2,     127,      86,      62,     102,       4,     124,
           105,      17,     456,      28,     328,      38,      25,
           480,      47,      56,      53,     312,       1,      27,
            46,     

Удаляем строки с выбросами в часах работы за месяц. Число не может быть больше чем 40320, если в месяце 28 дней и 43200, если 30 дней. У нас всего один выброс

In [72]:
df = df.loc[df['WORK_TIME'] != 2867959]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15221 entries, 0 to 22311
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID_CLIENT                15221 non-null  int64 
 1   AGE                      15221 non-null  int64 
 2   GENDER                   15221 non-null  int64 
 3   EDUCATION                15221 non-null  object
 4   MARITAL_STATUS           15221 non-null  object
 5   CHILD_TOTAL              15221 non-null  int64 
 6   DEPENDANTS               15221 non-null  int64 
 7   SOCSTATUS_WORK_FL        15221 non-null  int64 
 8   SOCSTATUS_PENS_FL        15221 non-null  int64 
 9   REG_ADDRESS_PROVINCE     15221 non-null  object
 10  FACT_ADDRESS_PROVINCE    15221 non-null  object
 11  POSTAL_ADDRESS_PROVINCE  15221 non-null  object
 12  FL_PRESENCE_FL           15221 non-null  int64 
 13  OWN_AUTO                 15221 non-null  int64 
 14  AGREEMENT_RK             15221 non-null  in

In [73]:
df['EDUCATION'].value_counts()

EDUCATION
Среднее специальное               6517
Среднее                           4678
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: count, dtype: int64

In [74]:
df['MARITAL_STATUS'].value_counts()

MARITAL_STATUS
Состою в браке        9415
Не состоял в браке    3621
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: count, dtype: int64

In [75]:
df['FAMILY_INCOME'].value_counts()

FAMILY_INCOME
от 10000 до 20000 руб.    7077
от 20000 до 50000 руб.    6080
от 5000 до 10000 руб.     1512
свыше 50000 руб.           492
до 5000 руб.                60
Name: count, dtype: int64

Для обучения мождели можно было бы закодировать категориальные признаки, как показано ниже. В данном случае я решил этого не делать, чтобы было проще выводить данные при визуализации.

In [76]:
def education(val):
    mapping ={
        'Среднее специальное': 0, 'Среднее':1, 'Высшее':2, 'Неоконченное высшее':3,
       'Неполное среднее':4, 'Два и более высших образования':5, 'Ученая степень':6
    }
    return mapping[val]
def marital_status(val):
    mapping = {
        'Состою в браке': 0, 'Не состоял в браке': 1, 'Разведен(а)': 2, 'Вдовец/Вдова': 3,
       'Гражданский брак': 4
    }
    return mapping[val]

def income(val):
    mapping = {
        'от 10000 до 20000 руб.': 0, 'от 20000 до 50000 руб.': 1,
       'от 5000 до 10000 руб.': 2, 'свыше 50000 руб.': 3, 'до 5000 руб.': 4
    }
    return mapping[val]

In [77]:
#df['EDUCATION'] = df['EDUCATION'].apply(education)
#df['MARITAL_STATUS'] = df['MARITAL_STATUS'].apply(marital_status)
#df['FAMILY_INCOME'] = df['FAMILY_INCOME'].apply(income)

In [78]:
#df_encoded = pd.get_dummies(df, columns=['EDUCATION', 'MARITAL_STATUS', 'FAMILY_INCOME'])
#df_encoded

In [79]:
df.columns

Index(['ID_CLIENT', 'AGE', 'GENDER', 'EDUCATION', 'MARITAL_STATUS',
       'CHILD_TOTAL', 'DEPENDANTS', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
       'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE',
       'POSTAL_ADDRESS_PROVINCE', 'FL_PRESENCE_FL', 'OWN_AUTO', 'AGREEMENT_RK',
       'TARGET', 'WORK_TIME', 'CREDIT', 'TERM', 'FST_PAYMENT',
       'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'FAMILY_INCOME',
       'PERSONAL_INCOME'],
      dtype='object')

In [80]:
df

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,AGREEMENT_RK,TARGET,WORK_TIME,CREDIT,TERM,FST_PAYMENT,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,FAMILY_INCOME,PERSONAL_INCOME
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,...,60099204,1,3,5588,6,1000,1,0,от 20000 до 50000 руб.,25000
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,...,62244665,0,5,19498,12,0,1,0,от 10000 до 20000 руб.,10000
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,...,61050759,0,360,15470,3,15000,1,1,от 20000 до 50000 руб.,30000
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,...,62079659,0,3,13960,6,2500,1,0,от 20000 до 50000 руб.,25000
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,...,66583553,0,12,11890,6,8000,2,2,от 10000 до 20000 руб.,15000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,...,66751099,0,36,11750,4,1900,2,2,от 5000 до 10000 руб.,8000
22308,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,...,64562377,0,24,12350,6,1380,1,0,от 20000 до 50000 руб.,12000
22309,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,...,62236542,0,36,4915,10,2000,1,0,от 5000 до 10000 руб.,9000
22310,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,...,66739926,0,36,5860,3,2000,1,1,от 10000 до 20000 руб.,12000


Сохраняем наш датафрейм в итоговый файл CSV

In [81]:
df.to_csv('full_table.csv')